# With ChatModel

In [27]:
import json

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [28]:
def save_result(output, prompt_type, model, init, best_practice, format_prompt, output_parser, input_data, reruns):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice,
        'formatter_uri': format_prompt}
    result['output_parser'] = output_parser
    result['input_data'] = input_data
    result['reruns'] = reruns
    result['output'] = output

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [29]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

## Summary Length

In [ ]:
sample_name = "suggestion_sample.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [19]:
minChars = 39
maxChars = 70

init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'summaryLengthPrompt_V2.3.0.json'
bestPractice_prompt = load_prompt('prompts/summary/'+bestPractice_prompt_name).format(min=minChars, max=maxChars)

format_prompt_name = 'summaryLengthFormatPrompt_V1.0.0.json'
format_prompt = load_prompt('prompts/summary/'+format_prompt_name)

In [20]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [22]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

{'summary_old': 'Project Summaries',
 'summary_new': 'Single page display of project components and their issue statuses'}

In [23]:
print(len(output['summary_new']))
reruns = 0

66


In [26]:
if((len(output['summary_new']) > maxChars) | (len(output['summary_new']) < minChars)):
    print('Summary length is not within the expected range. Start a rerun.')
    reruns += 1
    output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
else:
    print('Summary length is within the expected range.')

Summary length is within the expected range.


In [25]:
save_result(output, 'summary', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name, reruns)

## Description Completness

In [ ]:
sample_name = "suggestion_sample.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [15]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'descriptionCompletnessPrompt_V1.0.0.json'
bestPractice_prompt = load_prompt('prompts/description/'+bestPractice_prompt_name).format()

format_prompt_name = 'descriptionCompletnessFormatPrompt_V1.0.0.json'
format_prompt = load_prompt('prompts/description/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [ ]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

In [18]:
save_result(output, 'description', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name, 0)

## Field Update

maybe try Few-Shot here for better results??

In [76]:
sample_name = "suggestion_sample.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [77]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'updatePrompt_V1.16.0.json'
bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format()

format_prompt_name = 'updateFormatPrompt_V1.3.0.json'
format_prompt = load_prompt('prompts/update/'+format_prompt_name)

In [78]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [79]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})

In [80]:
save_result(output, 'update', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name, 0)

## Toxic Speech Detection

In [60]:
sample_name = "last_row_toxic.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [61]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

format_prompt_name = 'toxicSpeechFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/toxicSpeech/'+format_prompt_name)

In [62]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [63]:
output = await chain2.ainvoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

{'comment': 'Issue was fixed in latest Sourcetree versions you idiot. Use your brain and use Sourcetree 3.4.5 >',
 'author': 'Oleksandr Naumenko',
 'datetime': '2021-11-18 15:12:59.861000+00:00',
 'recommendation': 'The issue has been resolved in the latest Sourcetree versions. I recommend using Sourcetree 3.4.5 or later.',
 'reason': 'The comment contains toxic speech and should be updated to maintain a respectful and professional tone. The author should avoid using derogatory terms and instead focus on providing helpful information.'}

In [65]:
save_result(output, 'toxicSpeech', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name, 0)

## Bug Report Structure

In [48]:
sample_name = "desc_struc_last_row.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [49]:
bugReportStrucutre_prompt_name = 'bugReportStructurePrompt_V2.3.0.json'
bugReportStrucutre_prompt = load_prompt('prompts/bugReportStructure/'+bugReportStrucutre_prompt_name)

In [50]:
chain = bugReportStrucutre_prompt | model | StrOutputParser()

In [51]:
output = chain.invoke({"bug_report":json_sample})

Why data fromdescription field is taken into account here?

In [53]:
output = output.replace('\\n', '\n').replace('\\t', '\t')
print(output)


1. Overview
It is not possible to push if force push flag is activated. Nothing happens afterwards, there is an exception in log.

2. Steps to reproduce
- Activate force push flag
- Attempt to push

3. Actual result
Nothing happens, there is an exception in log.

4. Expected result
Push should be successful even with force push flag activated.

5. Stack Traces
ERROR [2021-04-12 10:15:45,362] [31] [SourceTree.ViewModel.RepoDialogContainer] [Log] - Wątkiem wywołującym musi być STA, ponieważ wiele składników interfejsów użytkownika go wymaga.
System.InvalidOperationException: Wątkiem wywołującym musi być STA, ponieważ wiele składników interfejsów użytkownika go wymaga.
w System.Windows.Input.InputManager..ctor()
w System.Windows.Input.InputManager.GetCurrentInputManagerImpl()
w System.Windows.Input.KeyboardNavigation..ctor()
w System.Windows.FrameworkElement.FrameworkServices..ctor()
w System.Windows.FrameworkElement.EnsureFrameworkServices()
w System.Windows.FrameworkElement..ctor()
w Sy

In [54]:
save_result(output, 'bugReportStructure', model_name, '-', bugReportStrucutre_prompt_name, '-', 'StrOutputParser', sample_name, 0)

## Internationalization

In [30]:
sample_name = "last_row_german.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [35]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'internationalizationPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/internationalization/'+bestPractice_prompt_name).format()

format_prompt_name = 'internationalizationFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/internationalization/'+format_prompt_name)

In [36]:
chain1 = init_prompt | model | StrOutputParser()

chain2 = (
    {"revised_ticket":chain1}
    | format_prompt
    | model
    | JsonOutputParser()
    )

In [37]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

{'Summary': 'Sourcetree crashes',
 'Description': 'The Sourcetree crashes every time I open it, I have uninstalled and installed several times',
 'VersionsAffected': '3.4.4',
 'IssueType': 'Bug',
 'Project': 'Sourcetree for Windows',
 'Components': 'Git',
 'CreatedDate': '2021-09-13T05:48:08.000+0000',
 'ResolvedDate': '2021-11-18T15:12:59.000+0000',
 'Status': 'Closed',
 'Priority': 'Low',
 'Creator': 'Shefali Bhandary',
 'Reporter': 'Shefali Bhandary',
 'Resolution': 'Cannot Reproduce',
 'IssueLinks': None,
 'Labels': None,
 'VersionsFixed': None,
 'Assignee': None,
 'TimeSpent': None,
 'Comments': [{'Author': 'Vipin Yadav',
   'Created': '2021-09-13 06:12:40.315000+00:00',
   'Comment': 'Please try to install latest release of sourcetree 3.4.6 and let us know if you are still facing same issue.'},
  {'Author': 'Shefali Bhandary',
   'Created': '2021-09-13 06:56:53.753000+00:00',
   'Comment': 'In our software portal on 3.2.6 is available'},
  {'Author': 'Vipin Yadav',
   'Created': 

In [38]:
save_result(output, 'internationalization', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name, 0)